In [ ]:
import sys
sys.path.append('..')

from zo_signsgd import ZOSignSGDttack
from simba import SimBAttack
from nes import NESAttack
from hsja import HSJAttack
from joint_attack import JointAttack

# Helper imports
from data import CIFAR10, IMAGENETTE
import utils

# Numerical computing and display imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Other imports
from datetime import datetime
import os

In [ ]:
# Load and Prepare the Data
folder_path = os.path.sep.join(['..', '..', 'images/'])
dataset = 'imagenette'    
if dataset == 'cifar10':
    images = CIFAR10()
if dataset == 'imagenette':
    images = IMAGENETTE()
x = images.load()
images.save_to_disk(x, folder_path, num_images=100)
del(x)

In [ ]:
hash_func = 'pdq'
hamm_thresholds = [0.1, 0.2, 0.3, 0.4]
output_imgs = []

for i in range(len(hamm_thresholds)):
    # Threshold-hyperparameter mappings
    if hash_func == 'neuralhash':
        attack_hamming_threshold = int(hamm_thresholds[i] * 128)

        threshold_hyperparam = {
            '1':  {'simba_hamming_threshold': 7,  'simba_l2_threshold': 20, 'hsja_l2_threshold': 8},
            '6':  {'simba_hamming_threshold': 10, 'simba_l2_threshold': 20, 'hsja_l2_threshold': 15},
            '12': {'simba_hamming_threshold': 20, 'simba_l2_threshold': 28, 'hsja_l2_threshold': 15},
            '25': {'simba_hamming_threshold': 25, 'simba_l2_threshold': 30, 'hsja_l2_threshold': 15},
            '38': {'simba_hamming_threshold': 40, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 15},
            '51': {'simba_hamming_threshold': 55, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 15}
        }

        nes_threshold_hyperparam = {
            '1':  {'nes_l2_threshold': 25, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 10},
            '6':  {'nes_l2_threshold': 35, 'nes_l2_tolerance': 12, 'hsja_l2_threshold': 15},
            '12': {'nes_l2_threshold': 45, 'nes_l2_tolerance': 15, 'hsja_l2_threshold': 20},
            '25': {'nes_l2_threshold': 55, 'nes_l2_tolerance': 20, 'hsja_l2_threshold': 24},
            '38': {'nes_l2_threshold': 75, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24},
            '51': {'nes_l2_threshold': 100, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24}
        }

        zo_threshold_hyperparam = {
            '1':  {'zo_l2_threshold': 25, 'zo_l2_tolerance': 10},
            '6':  {'zo_l2_threshold': 35, 'zo_l2_tolerance': 12},
            '12': {'zo_l2_threshold': 45, 'zo_l2_tolerance': 15},
            '25': {'zo_l2_threshold': 55, 'zo_l2_tolerance': 20},
            '38': {'zo_l2_threshold': 75, 'zo_l2_tolerance': 10},
            '51': {'zo_l2_threshold': 100, 'zo_l2_tolerance': 10}
        }  
    elif hash_func == 'pdq':
        attack_hamming_threshold = int(hamm_thresholds[i] * 256)

        # PDQ
        threshold_hyperparam = {
            '2':  {'simba_hamming_threshold': 7,  'simba_l2_threshold': 20, 'hsja_l2_threshold': 8},
            '12':  {'simba_hamming_threshold': 20, 'simba_l2_threshold': 20, 'hsja_l2_threshold': 12},
            '25': {'simba_hamming_threshold': 40, 'simba_l2_threshold': 28, 'hsja_l2_threshold': 15},
            '51': {'simba_hamming_threshold': 50, 'simba_l2_threshold': 30, 'hsja_l2_threshold': 15},
            '76': {'simba_hamming_threshold': 80, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 15},
            '102': {'simba_hamming_threshold': 110, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 15}
        }

        nes_threshold_hyperparam = {
            '2':  {'nes_l2_threshold': 25, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 10},
            '12':  {'nes_l2_threshold': 35, 'nes_l2_tolerance': 12, 'hsja_l2_threshold': 15},
            '25': {'nes_l2_threshold': 45, 'nes_l2_tolerance': 15, 'hsja_l2_threshold': 20},
            '51': {'nes_l2_threshold': 55, 'nes_l2_tolerance': 20, 'hsja_l2_threshold': 24},
            '76': {'nes_l2_threshold': 75, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24},
            '102': {'nes_l2_threshold': 100, 'nes_l2_tolerance': 5, 'hsja_l2_threshold': 24}
        }

        zo_threshold_hyperparam = {
            '2':  {'zo_l2_threshold': 25, 'zo_l2_tolerance': 10},
            '12':  {'zo_l2_threshold': 35, 'zo_l2_tolerance': 12},
            '25': {'zo_l2_threshold': 45, 'zo_l2_tolerance': 15},
            '51': {'zo_l2_threshold': 55, 'zo_l2_tolerance': 20},
            '76': {'zo_l2_threshold': 75, 'zo_l2_tolerance': 10},
            '102': {'zo_l2_threshold': 100, 'zo_l2_tolerance': 10}
        } 
    else: # photodna
        attack_hamming_threshold = int(hamm_thresholds[i] * 1152)

        threshold_hyperparam = {
            '11':  {'simba_hamming_threshold': 7,  'simba_l2_threshold': 20, 'hsja_l2_threshold': 8},
            '57':  {'simba_hamming_threshold': 10, 'simba_l2_threshold': 20, 'hsja_l2_threshold': 15},
            '115': {'simba_hamming_threshold': 20, 'simba_l2_threshold': 28, 'hsja_l2_threshold': 15},
            '230': {'simba_hamming_threshold': 25, 'simba_l2_threshold': 30, 'hsja_l2_threshold': 15},
            '345': {'simba_hamming_threshold': 40, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 15},
            '460': {'simba_hamming_threshold': 55, 'simba_l2_threshold': 35, 'hsja_l2_threshold': 15}
        }

        nes_threshold_hyperparam = {
            '11':  {'nes_l2_threshold': 25, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 10},
            '57':  {'nes_l2_threshold': 35, 'nes_l2_tolerance': 12, 'hsja_l2_threshold': 15},
            '115': {'nes_l2_threshold': 45, 'nes_l2_tolerance': 15, 'hsja_l2_threshold': 20},
            '230': {'nes_l2_threshold': 55, 'nes_l2_tolerance': 20, 'hsja_l2_threshold': 24},
            '345': {'nes_l2_threshold': 100, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24},
            '460': {'nes_l2_threshold': 100, 'nes_l2_tolerance': 10, 'hsja_l2_threshold': 24}
        }

        zo_threshold_hyperparam = {
            '11':  {'zo_l2_threshold': 25, 'zo_l2_tolerance': 10},
            '57':  {'zo_l2_threshold': 35, 'zo_l2_tolerance': 12},
            '115': {'zo_l2_threshold': 45, 'zo_l2_tolerance': 15},
            '230': {'zo_l2_threshold': 55, 'zo_l2_tolerance': 20},
            '345': {'zo_l2_threshold': 75, 'zo_l2_tolerance': 10},
            '460': {'zo_l2_threshold': 100, 'zo_l2_tolerance': 10}
        }  

    # SIMBA Hyperparams
    simba_epsilon = 0.9
    simba_hamming_threshold = threshold_hyperparam[str(attack_hamming_threshold)]['simba_hamming_threshold']                                # <= 5% - 10(max) & 10% - 20 & 20% - 23 
    simba_l2_threshold = threshold_hyperparam[str(attack_hamming_threshold)]['simba_l2_threshold']
    simba_max_steps = 1500
    fast = True

    # HSJA Hyperparams
    hsja_max_steps = 10
    hsja_grad_queries = 20
    hsja_l2_threshold = threshold_hyperparam[str(attack_hamming_threshold)]['hsja_l2_threshold']

    # Other params
    ssim_threshold = 0.8
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")
        
    # NES Hyperparams
    nes_mean = -0.025   #-0.1 (PDQ)
    nes_std = 0.1       #0.001 (PDQ)
    nes_sigma = 0.7
    nes_eps = 0.1                      
    nes_l2_threshold = nes_threshold_hyperparam[str(attack_hamming_threshold)]['nes_l2_threshold'] 
    nes_l2_tolerance = nes_threshold_hyperparam[str(attack_hamming_threshold)]['nes_l2_tolerance'] 
    nes_num_samples = 50

    # ZO-SignSGD Hyperparameters
    zo_max_queries = 50
    zo_epsilon = 0.1
    zo_upper_tolerance = 10 
    zo_lower_tolerance = 5
    zo_l2_threshold = zo_threshold_hyperparam[str(attack_hamming_threshold)]['zo_l2_threshold'] 
    zo_l2_tolerance = zo_threshold_hyperparam[str(attack_hamming_threshold)]['zo_l2_tolerance'] 
    zo_search_steps = 50

    # Soft-Label Attacks 
    zosignsgd = ZOSignSGDttack(max_queries=zo_max_queries, 
                            epsilon=zo_epsilon, 
                            l2_threshold=zo_l2_threshold,
                            l2_tolerance=zo_l2_tolerance,
                            search_steps=zo_search_steps)
    simba = SimBAttack(eps=simba_epsilon,
                    hamming_threshold=simba_hamming_threshold,
                    l2_threshold=simba_l2_threshold,
                    max_steps=simba_max_steps,
                    fast=fast)
    nes = NESAttack(mean=nes_mean,
                    std=nes_std,
                    sigma=nes_sigma,
                    eps=nes_eps,
                    l2_threshold=nes_l2_threshold,
                    l2_tolerance=nes_l2_tolerance,
                    num_samples=nes_num_samples)

    # Hard-label Attack import
    hsja = HSJAttack(max_iters=hsja_max_steps, 
                    grad_queries=hsja_grad_queries, 
                    l2_threshold=hsja_l2_threshold, 
                    hamming_threshold=attack_hamming_threshold)

    # Joint Attack
    joint_attack = JointAttack(soft_label=nes, 
                            hard_label=hsja)

    img_path = os.path.sep.join(['..', '..', 'images', '62.bmp'])
    _, _, _, _, path, filetype = img_path.split('.')
    img_path = path.split('/')
    img_path = f'{folder_path}{img_path[2]}.{filetype}'

    orig_img, sl_img, adv_img, sl_num_queries, hl_num_queries = joint_attack.attack(img_path, hash_func=hash_func)
    output_imgs.append(adv_img)

In [ ]:
plt.imshow(orig_img)
plt.axis('off')

In [ ]:
plt.figure(figsize=(20, 15))

plt.subplot(4, 1, 1)
plt.imshow(output_imgs[0].astype(np.uint8))
#plt.title('Adv Image (20%)')
plt.axis('off')

plt.subplot(4, 1, 2)
plt.imshow(output_imgs[1].astype(np.uint8))
#plt.title('Adv Image (30%)')
plt.axis('off')

plt.subplot(4, 1, 3)
plt.imshow(output_imgs[2].astype(np.uint8))
#plt.title('Adv Image (40%)')
plt.axis('off')

plt.subplot(4, 1, 4)
plt.imshow(output_imgs[3].astype(np.uint8))
#plt.title('Adv Image (40%)')
plt.axis('off')

plt.show()

In [ ]:
if hash_func == 'neuralhash':
    normalizing_factor = 128 
elif hash_func == 'pdq':
    normalizing_factor = 256
else: # photodna
    normalizing_factor = 1152
    
for i in range(4):
    orig_hash, target_hash = utils.compute_hash(img_path, hash_func='pdq'), utils.compute_hash(output_imgs[i], hash_func='pdq')
    hamm_dist = utils.distance(orig_hash, target_hash, 'hamming', 'pdq')
    l2_dist = utils.distance(orig_img, output_imgs[i], 'l2')
    ssim = utils.distance(orig_img.astype(np.uint8), output_imgs[i].astype(np.uint8), 'ssim')
    print(f'Relative Hamming Distance: {hamm_dist/normalizing_factor}\nL2 Dist: {l2_dist/np.sqrt(224*224*3)}\nSSIM: {ssim}\n')